# CS 595A Final Project
## Sudoku Solver

##### Jonathon Khi Franse, Shubham Nipanikar, Dhruvil Vekariya

This program is designed to receive a user-inputted Sudoku puzzle of any difficulty level and solve it using a Greedy Best-First Search algorithm. While the heuristic was designed to be condition-based, as opposed to being based on a traditional mathematical function, there is still a clear fitness hierarchy to which the algorithm adheres.

No external resources or third-party code was utilized in the creation of this code.

### Puzzle Input

In [8]:
# Initialize puzzle.

# "puzzle" represents the Sudoku puzzle itself. "puzzle" will be passed to the main algorithm,
# where it will be manipulated until a solution is found. The zeros represent blank spaces.

# Choose an example puzzle below to run through the solver by uncommenting it,
# or uncomment the Custom Puzzle at the bottom and modify it to input your own puzzle.


# Easy Test Case

# puzzle = [[1,0,0,0,9,0,7,0,3],
#           [0,7,6,0,0,8,4,0,0],
#           [9,3,0,0,7,0,0,0,5],
#           [3,0,7,0,0,6,2,0,8],
#           [0,1,0,0,0,0,0,7,0],
#           [8,0,2,5,0,0,9,0,1],
#           [4,0,0,0,6,0,0,3,7],
#           [0,0,3,4,0,0,8,9,0],
#           [5,0,9,0,8,0,0,0,2]]


# Medium Test Cases

# puzzle = [[0,7,0,8,0,0,0,0,2],
#           [9,4,6,0,0,0,0,8,0],
#           [0,0,0,6,9,5,0,0,0],
#           [0,5,8,0,2,0,7,0,0],
#           [0,0,0,4,6,8,0,0,0],
#           [0,0,2,0,5,0,8,4,0],
#           [0,0,0,2,7,9,0,0,0],
#           [0,9,0,0,0,0,2,7,3],
#           [5,0,0,0,0,1,0,9,0]]

# puzzle = [[0,0,1,5,0,0,3,0,0],
#           [0,0,0,0,1,3,0,0,7],
#           [3,5,0,0,0,7,9,0,0],
#           [0,0,0,0,6,0,0,3,2],
#           [0,0,8,0,0,0,6,0,0],
#           [2,6,0,0,8,0,0,0,0],
#           [0,0,4,7,0,0,0,2,6],
#           [5,0,0,2,3,0,0,0,0],
#           [0,0,2,0,0,4,5,0,0]]


# Hard Test Case

# puzzle = [[8,7,0,0,0,0,0,0,3],
#           [0,9,0,0,0,8,0,0,0],
#           [5,0,0,2,0,0,0,0,4],
#           [0,0,6,0,1,0,0,7,0],
#           [0,0,0,3,0,7,0,0,0],
#           [0,0,0,8,0,0,5,6,0],
#           [0,0,5,4,0,0,0,0,0],
#           [2,0,1,0,0,3,7,4,0],
#           [0,0,0,0,0,1,0,0,2]]


# Expert Test Case

# puzzle = [[5,0,0,0,0,4,0,7,2],
#           [0,0,0,0,0,0,0,0,0],
#           [0,0,0,6,1,3,0,0,0],
#           [0,0,0,3,7,8,0,0,0],
#           [0,0,0,4,0,0,0,6,3],
#           [0,0,4,0,0,0,9,0,0],
#           [0,9,6,0,0,0,1,0,0],
#           [0,0,0,0,0,0,0,0,4],
#           [0,1,0,0,0,5,3,9,0]]


# Custom Puzzle

# puzzle = [[0,0,0,0,0,0,0,0,0],
#           [0,0,0,0,0,0,0,0,0],
#           [0,0,0,0,0,0,0,0,0],
#           [0,0,0,0,0,0,0,0,0],
#           [0,0,0,0,0,0,0,0,0],
#           [0,0,0,0,0,0,0,0,0],
#           [0,0,0,0,0,0,0,0,0],
#           [0,0,0,0,0,0,0,0,0],
#           [0,0,0,0,0,0,0,0,0]]


### Puzzle Initialization

In [9]:
# Initialize various utility matrices used by the algorithm.

# "num_blanks" is a value that represents the number of blank spaces
# currently present in the puzzle (represented by zeros).
num_blanks = sum([row.count(0) for row in puzzle])

# "values_potentials" is a 3-D list that stores every value that can possibly go in every blank space
# on the puzzle (filled spaces on the puzzle are left blank in "values_potentials", representing that
# there are no values that could potentially go in that space since the space is already filled).
values_potentials = [[[] for i in range(9)] for i in range(9)]

# "count_potentials" is similar to "values_potentials" in nature, but performs a different function.
# "count_potentials" is a 2-D list that records the number of potential values that can fill each
# blank space, rather than the potential values themselves.
count_potentials = [[0 for i in range(9)] for i in range(9)]

# "coordinates_fittest" is a 1-D list that temporarily stores information about the fittest
# blank space and value combination, as chosen by the heuristic procedure.
coordinates_fittest = []

# "coordinates_closed" is a 2-D list that records all of the paths explored by the algorithm.
# Each row represents a different path.
coordinates_closed = [[]]

# "values_closed" is a 2-D list that contains information about previously explored paths.
# This information prevents the algorithm from exploring paths that have already been
# been explored and found to be invalid.
values_closed = []

# "values_error" is a 1-D list that aids the algorithm in correcting errors by carrying information.
values_error = []

# "check_error" is a value that acts as a flag for error detection. If check_error = 0,
# no error was detected in the last iteration of the algorithm. If check_error = 1,
# then an error was detected and the puzzle must update "values_closed" and reinitialize
# "values_potentials" and "count_potentials". "check_error" starts in the error position
# so that the algorithm initializes "values_potentials" and "count_potentials" when it first begins.
check_error = 1

### Puzzle Algorithm

In [10]:
# Run the algorithm.

# While the puzzle is incomplete...
while num_blanks > 0:

    # If an error occurred, or if running the algorithm for the first time...
    if check_error == 1:
        
        # Determine invalid paths.
    
        # If more than one path...
        if len(coordinates_closed) > 1:
            # If the path beginning was incorrect...
            if coordinates_closed[-1] == []:
                for i in range(len(coordinates_closed)-1):
                    if len(coordinates_closed[i]) == 1:
                        # ... mark all explored path beginnings as invalid.
                        values_closed.append(coordinates_closed[i][0])
            # Otherwise, if there are other incorrect paths...
            else:
                for i in range(len(coordinates_closed)-1):
                    if coordinates_closed[-1] == coordinates_closed[i][:len(coordinates_closed[-1])]:
                        # ... mark them as invalid.
                        values_closed.append(coordinates_closed[i][len(coordinates_closed[-1])])

        # Determine all potential values for each cell.
    
        # For all rows...
        for i in range(9):
            # For all columns...
            for j in range(9):
                # If the cell is blank...
                if puzzle[i][j] == 0:
                    # Compare all numbers with those in the...
                    for k in range(9):
                        # ... same row...
                        if puzzle[i].count(k+1) == 0:
                            # ... same column...
                            if [puzzle[row][j] for row in range(9)].count(k+1) == 0:
                                # ... same 3x3 square...
                                if i < 3 and j < 3:
                                    if sum([puzzle[m][0:3].count(k+1) for m in range(3)]) == 0:
                                        # ... and the invalid path list...
                                        if values_closed.count([i,j,k+1]) == 0:
                                            # ... and record the potential values.
                                            values_potentials[i][j].append(k+1)
                                            count_potentials[i][j] += 1
                                elif i < 3 and j < 6:
                                    if sum([puzzle[m][3:6].count(k+1) for m in range(3)]) == 0:
                                        # ... and the invalid path list...
                                        if values_closed.count([i,j,k+1]) == 0:
                                            # ... and record the potential values.
                                            values_potentials[i][j].append(k+1)
                                            count_potentials[i][j] += 1
                                elif i < 3 and j < 9:
                                    if sum([puzzle[m][6:9].count(k+1) for m in range(3)]) == 0:
                                        # ... and the invalid path list...
                                        if values_closed.count([i,j,k+1]) == 0:
                                            # ... and record the potential values.
                                            values_potentials[i][j].append(k+1)
                                            count_potentials[i][j] += 1
                                elif i < 6 and j < 3:
                                    if sum([puzzle[m+3][0:3].count(k+1) for m in range(3)]) == 0:
                                        # ... and the invalid path list...
                                        if values_closed.count([i,j,k+1]) == 0:
                                            # ... and record the potential values.
                                            values_potentials[i][j].append(k+1)
                                            count_potentials[i][j] += 1
                                elif i < 6 and j < 6:
                                    if sum([puzzle[m+3][3:6].count(k+1) for m in range(3)]) == 0:
                                        # ... and the invalid path list...
                                        if values_closed.count([i,j,k+1]) == 0:
                                            # ... and record the potential values.
                                            values_potentials[i][j].append(k+1)
                                            count_potentials[i][j] += 1
                                elif i < 6 and j < 9:
                                    if sum([puzzle[m+3][6:9].count(k+1) for m in range(3)]) == 0:
                                        # ... and the invalid path list...
                                        if values_closed.count([i,j,k+1]) == 0:
                                            # ... and record the potential values.
                                            values_potentials[i][j].append(k+1)
                                            count_potentials[i][j] += 1
                                elif i < 9 and j < 3:
                                    if sum([puzzle[m+6][0:3].count(k+1) for m in range(3)]) == 0:
                                        # ... and the invalid path list...
                                        if values_closed.count([i,j,k+1]) == 0:
                                            # ... and record the potential values.
                                            values_potentials[i][j].append(k+1)
                                            count_potentials[i][j] += 1
                                elif i < 9 and j < 6:
                                    if sum([puzzle[m+6][3:6].count(k+1) for m in range(3)]) == 0:
                                        # ... and the invalid path list...
                                        if values_closed.count([i,j,k+1]) == 0:
                                            # ... and record the potential values.
                                            values_potentials[i][j].append(k+1)
                                            count_potentials[i][j] += 1
                                elif i < 9 and j < 9:
                                    if sum([puzzle[m+6][6:9].count(k+1) for m in range(3)]) == 0:
                                        # ... and the invalid path list...
                                        if values_closed.count([i,j,k+1]) == 0:
                                            # ... and record the potential values.
                                            values_potentials[i][j].append(k+1)
                                            count_potentials[i][j] += 1
                                            
        # Reset the invalid path list to prevent closing off valid paths later on.
        values_closed = []
    
    # Select the fittest cell using the heuristic conditions.

    # Check the puzzle for spaces with only one potential value.
    # For every row...
    for i in range(9):
        # If there is only one potential value for any space in the row...
        if 1 in count_potentials[i]:
            # ... flag the first valid space in the last valid row as the new fittest option...
            coordinates_fittest = [i,count_potentials[i].index(1)]
            # ... and include the potential value as part of the flag.
            coordinates_fittest.append(values_potentials[coordinates_fittest[0]][coordinates_fittest[1]][-1])
            
    # If none, check every row for any potential values that appear only once.
    if coordinates_fittest == []:
        # For every potential value...
        for k in range(9,0,-1):
                # For every row...
                for i in range(9):
                    # If the value is present...
                    if sum([values_potentials[i][m].count(k) for m in range(9)]) == 1:
                        # ... find the value...
                        for j in range(9):
                            if values_potentials[i][j].count(k) == 1:
                                # ... and flag the position and value as the new fittest option.
                                coordinates_fittest = [i,j,k]
    
    # If none, check every column for any potential values that appear only once.
    if coordinates_fittest == []:
        # For every potential value...
        for k in range(9,0,-1):
                # For every column...
                for j in range(9):
                    # If the value is present...
                    if sum([values_potentials[m][j].count(k) for m in range(9)]) == 1:
                        # ... find the value...
                        for i in range(9):
                            if values_potentials[i][j].count(k) == 1:
                                # ... and flag the position and value as the new fittest option.
                                coordinates_fittest = [i,j,k]
    
    # If none, check every 3x3 section for any potential values that appear only once.
    if coordinates_fittest == []:
        # For every potential value...
        for k in range(9,0,-1):
                # For every row...
                for i in range(3):
                    # If the value is present...
                    if sum([sum([values_potentials[(3*i)][m].count(k) for m in range(3)]),sum([values_potentials[(3*i)+1][n].count(k) for n in range(3)]),sum([values_potentials[(3*i)+2][o].count(k) for o in range(3)])]) == 1:
                        # ... find the value...
                        if sum([values_potentials[(3*i)][m].count(k for m in range(3))]) == 1:
                            for j in range(3):
                                if values_potentials[(3*i)][j].count(k) == 1:
                                    # ... and flag the position and value as the new fittest option.
                                    coordinates_fittest = [(3*i),j,k]
                        elif sum([values_potentials[(3*i)+1][m].count(k) for m in range(3)]) == 1:
                            for j in range(3):
                                if values_potentials[(3*i)+1][j].count(k) == 1:
                                    # ... and flag the position and value as the new fittest option.
                                    coordinates_fittest = [((3*i)+1),j,k]
                        elif sum([values_potentials[(3*i)+2][m].count(k) for m in range(3)]) == 1:
                            for j in range(3):
                                if values_potentials[(3*i)+2][j].count(k) == 1:
                                    # ... and flag the position and value as the new fittest option.
                                    coordinates_fittest = [((3*i)+2),j,k]
                    # If the value is present...
                    elif sum([sum([values_potentials[(3*i)][m+3].count(k) for m in range(3)]),sum([values_potentials[(3*i)+1][n+3].count(k) for n in range(3)]),sum([values_potentials[(3*i)+2][o+3].count(k) for o in range(3)])]) == 1:
                        # ... find the value...
                        if sum([values_potentials[(3*i)][m+3].count(k) for m in range(3)]) == 1:
                            for j in range(3):
                                if values_potentials[(3*i)][j+3].count(k) == 1:
                                    # ... and flag the position and value as the new fittest option.
                                    coordinates_fittest = [(3*i),(j+3),k]
                        elif sum([values_potentials[(3*i)+1][m+3].count(k) for m in range(3)]) == 1:
                            for j in range(3):
                                if values_potentials[(3*i)+1][j+3].count(k) == 1:
                                    # ... and flag the position and value as the new fittest option.
                                    coordinates_fittest = [((3*i)+1),(j+3),k]
                        elif sum([values_potentials[(3*i)+2][m+3].count(k) for m in range(3)]) == 1:
                            for j in range(3):
                                if values_potentials[(3*i)+2][j+3].count(k) == 1:
                                    # ... and flag the position and value as the new fittest option.
                                    coordinates_fittest = [((3*i)+2),(j+3),k]
                    # If the value is present...
                    elif sum([sum([values_potentials[(3*i)][m+6].count(k) for m in range(3)]),sum([values_potentials[(3*i)+1][n+6].count(k) for n in range(3)]),sum([values_potentials[(3*i)+2][o+6].count(k) for o in range(3)])]) == 1:
                        # ... find the value...
                        if sum([values_potentials[(3*i)][m+6].count(k) for m in range(3)]) == 1:
                            for j in range(3):
                                if values_potentials[(3*i)][j+6].count(k) == 1:
                                    # ... and flag the position and value as the new fittest option.
                                    coordinates_fittest = [(3*i),(j+6),k]
                        elif sum([values_potentials[(3*i)+1][m+6].count(k) for m in range(3)]) == 1:
                            for j in range(3):
                                if values_potentials[(3*i)+1][j+6].count(k) == 1:
                                    # ... and flag the position and value as the new fittest option.
                                    coordinates_fittest = [((3*i)+1),(j+6),k]
                        elif sum([values_potentials[(3*i)+2][m+6].count(k) for m in range(3)]) == 1:
                            for j in range(3):
                                if values_potentials[(3*i)+2][j+6].count(k) == 1:
                                    # ... and flag the position and value as the new fittest option.
                                    coordinates_fittest = [((3*i)+2),(j+6),k]
    
    # If none, check the puzzle for spaces with more than one potential values...
    if coordinates_fittest == []:
        # For every potential quantity of potential values...
        for k in range(9,1,-1):
            # For every row...
            for i in range(9):
                # If the quantity is present in the row...
                if k in count_potentials[i]:
                    # ... flag the first valid space in the last fittest row as the new fittest option...
                    coordinates_fittest = [i,count_potentials[i].index(k)]
                    # ... and include the potential value as part of the flag.
                    coordinates_fittest.append(values_potentials[coordinates_fittest[0]][coordinates_fittest[1]][-1])
    
    # Check to ensure that a valid cell was found.
    
    # If not, then an error was made. Revert to previous state. If no previous state, there was an error in the initial puzzle declaration.
    if coordinates_fittest == []:
        # Copy the current, invalid path to a new row, thus creating a new, identical path.
        coordinates_closed.append(coordinates_closed[-1].copy())
        # Remove the last addition to the path, which was proven to be invalid due to the error.
        values_error = coordinates_closed[-1].pop()
        # Reset the last cell to be modified in the puzzle, i.e. undo the path addition that was removed above.
        puzzle[values_error[0]][values_error[1]] = 0
        # Reset the potential values in the puzzle so that they can be found again in the next iteration.
        values_potentials = [[[] for i in range(9)] for i in range(9)]
        count_potentials = [[0 for i in range(9)] for i in range(9)]
        # Reset the list used to help correct the error.
        values_error = []
        # Set the error flag equal to 1, signifying that an error has been made.
        check_error = 1

    # If so, then modify the selected cell.
    else:
        # Add cell coordinates to closed list.
        coordinates_closed[-1].append(coordinates_fittest)
        # Replace blank cell with the selected potential value.
        puzzle[coordinates_fittest[0]][coordinates_fittest[1]] = coordinates_fittest[2]
        # Empty the potential values for the cell that was just filled.
        values_potentials[coordinates_fittest[0]][coordinates_fittest[1]] = []
        count_potentials[coordinates_fittest[0]][coordinates_fittest[1]] = []
        
        # Adjust relevant rows, columns, and 3x3 sections.
        
        for i in range(9):
            # For every blank cell in the row...
            if puzzle[coordinates_closed[-1][-1][0]][i] == 0:
                # If the value just placed is a potential value in the selected cell...
                if values_potentials[coordinates_closed[-1][-1][0]][i].count(coordinates_closed[-1][-1][2]) > 0:
                    # ... remove the value from the cell's potential values.
                    values_potentials[coordinates_closed[-1][-1][0]][i].remove(coordinates_closed[-1][-1][2])
                    count_potentials[coordinates_closed[-1][-1][0]][i] -= 1
            # For every blank cell in the column...
            if puzzle[i][coordinates_closed[-1][-1][1]] == 0:
                # If the value just placed is a potential value in the selected cell...
                if values_potentials[i][coordinates_closed[-1][-1][1]].count(coordinates_closed[-1][-1][2]) > 0:
                    # ... remove the value from the cell's potential values.
                    values_potentials[i][coordinates_closed[-1][-1][1]].remove(coordinates_closed[-1][-1][2])
                    count_potentials[i][coordinates_closed[-1][-1][1]] -= 1
        
        # Find the 3x3 section where the modified cell is located.
        
        if coordinates_closed[-1][-1][0] < 3 and coordinates_closed[-1][-1][1] < 3:
            # For every blank cell in the 3x3 section...
            for i in range(3):
                for j in range(3):
                    if puzzle[i][j] == 0:
                        # If the value just placed is a potential value in the selected cell...
                        if values_potentials[i][j].count(coordinates_closed[-1][-1][2]) > 0:
                            # ... remove the value from the cell's potential values.
                            values_potentials[i][j].remove(coordinates_closed[-1][-1][2])
                            count_potentials[i][j] -= 1
        elif coordinates_closed[-1][-1][0] < 3 and coordinates_closed[-1][-1][1] < 6:
            # For every blank cell in the 3x3 section...
            for i in range(3):
                for j in range(3):
                    if puzzle[i][j+3] == 0:
                        # If the value just placed is a potential value in the selected cell...
                        if values_potentials[i][j+3].count(coordinates_closed[-1][-1][2]) > 0:
                            values_potentials[i][j+3].remove(coordinates_closed[-1][-1][2])
                            count_potentials[i][j+3] -= 1
        elif coordinates_closed[-1][-1][0] < 3 and coordinates_closed[-1][-1][1] < 9:
            # For every blank cell in the 3x3 section...
            for i in range(3):
                for j in range(3):
                    if puzzle[i][j+6] == 0:
                        # If the value just placed is a potential value in the selected cell...
                        if values_potentials[i][j+6].count(coordinates_closed[-1][-1][2]) > 0:
                            # ... remove the value from the cell's potential values.
                            values_potentials[i][j+6].remove(coordinates_closed[-1][-1][2])
                            count_potentials[i][j+6] -= 1
        elif coordinates_closed[-1][-1][0] < 6 and coordinates_closed[-1][-1][1] < 3:
            # For every blank cell in the 3x3 section...
            for i in range(3):
                for j in range(3):
                    if puzzle[i+3][j] == 0:
                        # If the value just placed is a potential value in the selected cell...
                        if values_potentials[i+3][j].count(coordinates_closed[-1][-1][2]) > 0:
                            # ... remove the value from the cell's potential values.
                            values_potentials[i+3][j].remove(coordinates_closed[-1][-1][2])
                            count_potentials[i+3][j] -= 1
        elif coordinates_closed[-1][-1][0] < 6 and coordinates_closed[-1][-1][1] < 6:
            # For every blank cell in the 3x3 section...
            for i in range(3):
                for j in range(3):
                    if puzzle[i+3][j+3] == 0:
                        # If the value just placed is a potential value in the selected cell...
                        if values_potentials[i+3][j+3].count(coordinates_closed[-1][-1][2]) > 0:
                            # ... remove the value from the cell's potential values.
                            values_potentials[i+3][j+3].remove(coordinates_closed[-1][-1][2])
                            count_potentials[i+3][j+3] -= 1
        elif coordinates_closed[-1][-1][0] < 6 and coordinates_closed[-1][-1][1] < 9:
            # For every blank cell in the 3x3 section...
            for i in range(3):
                for j in range(3):
                    if puzzle[i+3][j+6] == 0:
                        # If the value just placed is a potential value in the selected cell...
                        if values_potentials[i+3][j+6].count(coordinates_closed[-1][-1][2]) > 0:
                            # ... remove the value from the cell's potential values.
                            values_potentials[i+3][j+6].remove(coordinates_closed[-1][-1][2])
                            count_potentials[i+3][j+6] -= 1
        elif coordinates_closed[-1][-1][0] < 9 and coordinates_closed[-1][-1][1] < 3:
            # For every blank cell in the 3x3 section...
            for i in range(3):
                for j in range(3):
                    if puzzle[i+6][j] == 0:
                        # If the value just placed is a potential value in the selected cell...
                        if values_potentials[i+6][j].count(coordinates_closed[-1][-1][2]) > 0:
                            # ... remove the value from the cell's potential values.
                            values_potentials[i+6][j].remove(coordinates_closed[-1][-1][2])
                            count_potentials[i+6][j] -= 1
        elif coordinates_closed[-1][-1][0] < 9 and coordinates_closed[-1][-1][1] < 6:
            # For every blank cell in the 3x3 section...
            for i in range(3):
                for j in range(3):
                    if puzzle[i+6][j+3] == 0:
                        # If the value just placed is a potential value in the selected cell...
                        if values_potentials[i+6][j+3].count(coordinates_closed[-1][-1][2]) > 0:
                            # ... remove the value from the cell's potential values.
                            values_potentials[i+6][j+3].remove(coordinates_closed[-1][-1][2])
                            count_potentials[i+6][j+3] -= 1
        elif coordinates_closed[-1][-1][0] < 9 and coordinates_closed[-1][-1][1] < 9:
            # For every blank cell in the 3x3 section...
            for i in range(3):
                for j in range(3):
                    if puzzle[i+6][j+6] == 0:
                        # If the value just placed is a potential value in the selected cell...
                        if values_potentials[i+6][j+6].count(coordinates_closed[-1][-1][2]) > 0:
                            # ... remove the value from the cell's potential values.
                            values_potentials[i+6][j+6].remove(coordinates_closed[-1][-1][2])
                            count_potentials[i+6][j+6] -= 1

        # Set the error flag equal to 0, signifying that no error has been made.
        check_error = 0

    # Reset fittest cell information.
    coordinates_fittest = []
    
    # Count remaining number of blanks.
    num_blanks = sum([row.count(0) for row in puzzle])

# When the puzzle is complete, tell the user.
display("Done!")

'Done!'

### Puzzle Output

In [11]:
# Display the outputs.

# Display the completed puzzle.
display("Completed Puzzle:")
display(puzzle)
# Display the number of paths taken. If only one path was taken, no errors occurred.
display("Number of Paths Taken:")
display(len(coordinates_closed))
# Display the final path taken. Each path step is displayed as [x-coordinate, y-coordinate, value chosen],
# with the top-left corner of the puzzle being position [0,0].
display("Final Path Taken:")
display(coordinates_closed[-1])

'Completed Puzzle:'

[[5, 3, 1, 9, 8, 4, 6, 7, 2],
 [6, 4, 9, 2, 5, 7, 8, 3, 1],
 [8, 2, 7, 6, 1, 3, 5, 4, 9],
 [9, 6, 2, 3, 7, 8, 4, 1, 5],
 [1, 8, 5, 4, 2, 9, 7, 6, 3],
 [3, 7, 4, 5, 6, 1, 9, 2, 8],
 [4, 9, 6, 8, 3, 2, 1, 5, 7],
 [7, 5, 3, 1, 9, 6, 2, 8, 4],
 [2, 1, 8, 7, 4, 5, 3, 9, 6]]

'Number of Paths Taken:'

1

'Final Path Taken:'

[[0, 2, 1],
 [0, 1, 3],
 [5, 0, 3],
 [6, 4, 3],
 [7, 2, 3],
 [1, 7, 3],
 [1, 8, 1],
 [3, 8, 5],
 [6, 0, 4],
 [8, 4, 4],
 [6, 7, 5],
 [7, 1, 5],
 [2, 6, 5],
 [8, 8, 6],
 [0, 6, 6],
 [4, 2, 5],
 [2, 8, 9],
 [7, 7, 8],
 [6, 8, 7],
 [7, 6, 2],
 [7, 0, 7],
 [6, 5, 2],
 [6, 3, 8],
 [8, 3, 7],
 [5, 8, 8],
 [4, 6, 7],
 [3, 6, 4],
 [2, 7, 4],
 [1, 6, 8],
 [0, 3, 9],
 [7, 3, 1],
 [1, 5, 7],
 [0, 4, 8],
 [1, 1, 4],
 [1, 0, 6],
 [3, 1, 6],
 [5, 1, 7],
 [2, 2, 7],
 [1, 2, 9],
 [3, 2, 2],
 [8, 2, 8],
 [8, 0, 2],
 [4, 1, 8],
 [3, 7, 1],
 [5, 7, 2],
 [5, 3, 5],
 [5, 4, 6],
 [7, 4, 9],
 [7, 5, 6],
 [5, 5, 1],
 [4, 4, 2],
 [4, 5, 9],
 [4, 0, 1],
 [3, 0, 9],
 [2, 0, 8],
 [2, 1, 2],
 [1, 3, 2],
 [1, 4, 5]]